In [256]:
from IPython import get_ipython

import numpy as np
from numpy import random, dtype
from array import array
import math
from math import floor

import matplotlib

# matplotlib.use('agg')
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib widget
# %matplotlib widget
# %matplotlib widget

from os import path, mkdir
from tqdm import tqdm

import ROOT
from ROOT import TCanvas, TFile, TPaveText, TMath, TF1, TH1, TH1F, TH2D, TH2F, TH3F, TRandom, TPolyMarker3D, TGraphErrors, TBox, TLine
from ROOT import TSpline5
from ROOT import gROOT, gBenchmark, gRandom, gInterpreter, gPad, gStyle, gDirectory
from ROOT import nullptr, kBlue, kRed, kGreen, kBlack, kMagenta, kOrange

# import rootpy
%jsroot on

In [257]:
ROOT.gROOT.LoadMacro('includes/rootlogon.C')

0


 Beginning new ROOT session with private TStyle 


In [258]:
canvasCnt = 0
canvases = []

def NewCanvas(s1 = 1200, s2 = 800):
    
    global canvasCnt, canvases
    n = "" + str(canvasCnt)
    canvases.append(TCanvas(n, n, s1, s2))
    canvases[canvasCnt].Draw()
    canvasCnt = canvasCnt+1
    return canvasCnt-1, canvases[canvasCnt-1]

In [259]:
def FillGraph(gra: TGraphErrors, x, y, ex, ey):
    gra.AddPoint(x, y)
    n = gra.GetN()
    gra.SetPointError(n-1, ex, ey)
    return n

In [260]:
run = "run530164"
out = F"out/{run}/waveshapemcp/"
exten = ".pdf"
fin = TFile(F"../data/roottople_new/{run}_new.root")
tre = fin.Get("t")

try:
    mkdir(out)
except:
    pass

mtMcp = "( 0.5*(tim[2]+tim[3]) )"
dtMcp = "( tim[2]-tim[3] )"
mtSipm = "( 0.5*(tim[1]+tim[0]) )"
dtSipm = "( tim[1]-tim[0] )"
mcSipm = "( 0.5*(charge[1]+charge[0]) )"
asymSipm = "( (charge[1]-charge[0])/(charge[1]+charge[0]) )"
cutAnd = " && "

fidlarge = np.array([5.7, 6.7, 4.7, 5.6])
fidlarge = [5.7, 6.7, 4.7, 5.6] if run == "run530164" else fidlarge
fidlarge = [5.9, 7.0, 4.7, 5.6] if run == "run530167" else fidlarge 
fidlarge = [5.9, 7.0, 4.5, 5.5] if run == "run530181" else fidlarge
fidlarge = [5.9, 7.0, 4.5, 5.5] if run == "run530178" else fidlarge
fidLarge = F"cryPos[0]>{fidlarge[0]} && cryPos[0]<{fidlarge[1]} && cryPos[1]>{fidlarge[2]} && cryPos[1]<{fidlarge[3]}" 

cCutSipm = "charge[0] > 50 && charge[1] > 50"
histoWave = "(1900, -30, 160, 600, -0.1, 1.1)"

In [261]:
fidMidX = 0.5*(fidlarge[0]+fidlarge[1])
fidMidY = 0.5*(fidlarge[2]+fidlarge[3])

fidUL = F"cryPos[0]>{fidlarge[0]} && cryPos[0]<{fidMidX} && cryPos[1]>{fidMidY} && cryPos[1]<{fidlarge[3]}" 
fidUR = F"cryPos[0]>{fidMidX} && cryPos[0]<{fidlarge[1]} && cryPos[1]>{fidMidY} && cryPos[1]<{fidlarge[3]}" 
fidDL = F"cryPos[0]>{fidlarge[0]} && cryPos[0]<{fidMidX} && cryPos[1]>{fidlarge[2]} && cryPos[1]<{fidMidY}" 
fidDR = F"cryPos[0]>{fidMidX} && cryPos[0]<{fidlarge[1]} && cryPos[1]>{fidlarge[2]} && cryPos[1]<{fidMidY}" 
fidU = F"cryPos[0]>{fidlarge[0]} && cryPos[0]<{fidlarge[1]} && cryPos[1]>{fidMidY} && cryPos[1]<{fidlarge[3]}" 
fidD = F"cryPos[0]>{fidlarge[0]} && cryPos[0]<{fidlarge[1]} && cryPos[1]>{fidlarge[2]} && cryPos[1]<{fidMidY}" 
fidL = F"cryPos[0]>{fidlarge[0]} && cryPos[0]<{fidMidX} && cryPos[1]>{fidlarge[2]} && cryPos[1]<{fidlarge[3]}" 
fidR = F"cryPos[0]>{fidMidX} && cryPos[0]<{fidlarge[1]} && cryPos[1]>{fidlarge[2]} && cryPos[1]<{fidlarge[3]}" 

fidQuadrants = [fidUL, fidUR, fidDL, fidDR]
fidSides = [fidU, fidD, fidL, fidR]

In [262]:
tre.Draw(F"cryPos[1]:cryPos[0]>>hh(100, 3, 8, 100, 3, 8)",  cCutSipm , "goff")
hh = gDirectory.Get("hh")
_, cc = NewCanvas(500,300)
cc.SetLogz(0)
hh.SetStats(0)
hh.SetTitle(run)
hh.GetXaxis().SetTitle("track at cry X [cm]")
hh.GetYaxis().SetTitle("track at cry Y [cm]")
box = TBox(fidlarge[0], fidlarge[2], fidlarge[1], fidlarge[3])
box.SetLineColor(kRed)
box.SetLineWidth(3)
box.SetFillStyle(0)
l1, l2 = TLine(fidMidX, fidlarge[2], fidMidX, fidlarge[3]), TLine(fidlarge[0], fidMidY, fidlarge[1], fidMidY)
l1.SetLineWidth(3)
l2.SetLineWidth(3)
# l1.SetLineColor(kRed)
# l2.SetLineColor(kRed)
hh.Draw("zcol")
box.Draw("same")
l1.Draw("same")
l2.Draw("same")

In [263]:
_, cc = NewCanvas(600,300)
tre.Draw("charge[0]>>(600,0,600)")

In [264]:
qmin, qmax = 100, 180 
qCut = F"charge[0] > {qmin} && charge[1] > {qmin} && charge[0] < {qmax} && charge[1] < {qmax}"
qminq, qmax1 = 150, 200 
qCutCharge = F"charge[0] > {qmin} && charge[1] > {qmin} && charge[0] < {qmax} && charge[1] < {qmax}"

histoTime = "(400, 6, 8)"
histoQ = "(200,0.4,0.6)"
histoQm = "(200,-0.2,0.2)"

In [265]:
times1, times2, timesm = [],[],[]
q1, q2, qm = [],[],[]

for i in tqdm(range(4)):

    tre.Draw(F"tim[0]-{mtMcp}>>hhh{histoTime}", qCut + cutAnd + fidQuadrants[i], "zcol")
    times1.append(gDirectory.Get("hhh").Clone())
    tre.Draw(F"tim[1]-{mtMcp}>>hhh{histoTime}", qCut + cutAnd + fidQuadrants[i], "zcol")
    times2.append(gDirectory.Get("hhh").Clone())
    tre.Draw(F"{mtSipm}-{mtMcp}>>hhh{histoTime}", qCut + cutAnd + fidQuadrants[i], "zcol")
    timesm.append(gDirectory.Get("hhh").Clone())

    tre.Draw(F"0.5*charge[0]/{mcSipm}>>hhh{histoQ}", qCutCharge + cutAnd + fidQuadrants[i], "zcol")
    q1.append(gDirectory.Get("hhh").Clone())
    tre.Draw(F"0.5*charge[1]/{mcSipm}>>hhh{histoQ}", qCutCharge + cutAnd + fidQuadrants[i], "zcol")
    q2.append(gDirectory.Get("hhh").Clone())
    tre.Draw(F"{asymSipm}>>hhh{histoQm}", qCutCharge + cutAnd + fidQuadrants[i], "zcol")
    qm.append(gDirectory.Get("hhh").Clone())

100%|██████████| 4/4 [00:01<00:00,  2.81it/s]
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hhh (Potent

In [266]:
_, cc = NewCanvas(600,300)
cc.Divide(2,2)

for idx, (h1, h2) in enumerate(zip(times1, times2)):
    cc.cd(idx+1)
    cc.SetRightMargin(0.13)
    cc.SetLeftMargin(0.13)
    h1.SetStats(0)
    h1.GetXaxis().SetTitle("t_{i}- MCP_{mt} [ns]")
    h1.GetYaxis().SetTitle("Entries")
    h1.SetLineColor(kBlue)
    h1.GetYaxis().SetRangeUser(0,h1.GetMaximum()*1.2)
    h1.DrawClone()
    h2.SetLineColor(kBlack)
    h2.SetStats(0)
    h2.DrawClone("sames")

cc.SaveAs(out + "tChanQuadrants" + exten) 
# cc.SaveAs(out + "TimeSiPM1_MCP_PWO.pdf")

Info in <TCanvas::Print>: pdf file out/run530164/waveshapemcp/tChanQuadrants.pdf has been created


In [267]:
_, cc = NewCanvas(600,300)
cc.Divide(2,2)

for idx, hh in enumerate(timesm):
    cc.cd(idx+1)
    cc.SetRightMargin(0.13)
    cc.SetLeftMargin(0.13)
    hh.SetStats(1)
    hh.GetXaxis().SetTitle("t_{mean}- MCP_{mt} [ns]")
    hh.GetYaxis().SetTitle("Entries")
    hh.Fit("gaus", "EMQ", "")
    hh.DrawClone()

cc.SaveAs(out + "tMeanQuadrants" + exten) 
# cc.SaveAs(out + "TimeSiPM1_MCP_PWO.pdf")

Info in <TCanvas::Print>: pdf file out/run530164/waveshapemcp/tMeanQuadrants.pdf has been created


In [268]:
_, cc = NewCanvas(600,300)
cc.Divide(2,2)

for idx, (h1, h2) in enumerate(zip(q1, q2)):
    cc.cd(idx+1)
    cc.SetRightMargin(0.13)
    cc.SetLeftMargin(0.13)
    h1.SetStats(0)
    h1.GetXaxis().SetTitle("Qi / (Q1+Q2)")
    h1.GetYaxis().SetTitle("Entries")
    h1.SetLineColor(kBlue)
    h1.GetYaxis().SetRangeUser(0,h1.GetMaximum()*1.2)
    h1.DrawClone()
    h2.SetLineColor(kBlack)
    h2.SetStats(0)
    h2.DrawClone("sames")

cc.SaveAs(out + "q1Quad" + exten) 

Info in <TCanvas::Print>: pdf file out/run530164/waveshapemcp/q1Quad.pdf has been created


In [269]:
_, cc = NewCanvas(600,300)
cc.Divide(2,2)

for idx, hh in enumerate(qm):
    cc.cd(idx+1)
    cc.SetRightMargin(0.13)
    cc.SetLeftMargin(0.13)
    hh.SetStats(0)
    hh.GetXaxis().SetTitle("(Q2-Q1)/(Q1+Q2)")
    hh.GetYaxis().SetTitle("Entries")
    # hh.Fit("gaus", "EMQ", "")
    hh.DrawClone()

cc.SaveAs(out + "mqQuad" + exten) 

Info in <TCanvas::Print>: pdf file out/run530164/waveshapemcp/mqQuad.pdf has been created
